In [1]:
## This file implements neural networks and random forest on p0017presabs_qual.
## Due to the imbalanced data, we implement both over-sampling method and the combination of under- and over-sampling method.
## For fully-connected neural networks, the accuracy is 59.04% for over-sampling data, 61.45% after improvement.
## And it is 85.71% for combination data.
## To further improve the accuracy, we implement lasso regression to select important features, and thus have a new dataset.
## However, the new accuracy is 56.63% for over-sampling data and 59.04% after improvement.
## The new accuracy becomes 81.82% for combination data, and 86.36% after improvement.
## For random forest, the accuracy is 60.24% for over-sampling data and 85.71% for combination data.
## For random forest with cross-validation, the mean accuracy is 53.78% for over-sampling, and 93.46% for combination data.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p0017presabs_qual.csv')
df.shape

(255, 7004)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      2
1      0
2      2
3      2
4      2
5      2
6      2
7      0
8      1
9      1
10     1
11     2
12     2
13     1
14     0
15     0
16     2
17     2
18     0
19     0
20     1
21     0
22     0
23     2
24     0
25     1
26     0
27     0
28     1
29     0
      ..
225    1
226    2
227    2
228    0
229    2
230    1
231    1
232    2
233    2
234    0
235    1
236    0
237    0
238    2
239    0
240    0
241    0
242    0
243    2
244    2
245    0
246    0
247    2
248    0
249    1
250    2
251    1
252    0
253    2
254    2
Name: pheno, Length: 255, dtype: int64

In [6]:
df['pheno'].value_counts()

2    92
0    89
1    74
Name: pheno, dtype: int64

In [7]:
df.head()

,id,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTGATAATTT,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,...,group_315,group_381,group_426,group_4770,group_8913,group_9224,group_9493,group_9494,group_9905,pheno
0,107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,109,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
3,120335,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2
4,120337,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2


In [8]:
df_clean = df.drop(columns=['id'])

In [9]:
df_clean.shape

(255, 7003)

In [10]:
df_clean.head()

,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTGATAATTT,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,TTTTTTTATAACATTCAAAGTCTCACCATTGTCATTTGAATGATCATCAATAATAATTAATTCGTAATCAGTACTCTTCATTGTTTGATTTAATACAGAA,...,group_315,group_381,group_426,group_4770,group_8913,group_9224,group_9493,group_9494,group_9905,pheno
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2


In [11]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 7002) (255,)


In [12]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

Using TensorFlow backend.


[(0, 92), (1, 92), (2, 92)]


In [13]:
# combination of under- and over- sampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 21), (1, 16), (2, 30)]


In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [16]:
############# Fully-Connected Neural Network ################

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 193 samples, validate on 83 samples
Epoch 1/100
193/193 [==============================] - 0s 923us/step - loss: 1.2101 - accuracy: 0.4145 - val_loss: 1.6943 - val_accuracy: 0.4096
Epoch 2/100
193/193 [==============================] - 0s 272us/step - loss: 1.0555 - accuracy: 0.5389 - val_loss: 1.1141 - val_accuracy: 0.5663
Epoch 3/100
193/193 [==============================] - 0s 229us/step - loss: 1.0938 - accuracy: 0.6321 - val_loss: 1.0656 - val_accuracy: 0.5542
Epoch 4/100
193/193 [==============================] - 0s 449us/step - loss: 0.8767 - accuracy: 0.6114 - val_loss: 1.1532 - val_accuracy: 0.5181
Epoch 5/100
193/193 [==============================] - 0s 447us/step - loss: 0.7966 - accuracy: 0.6166 - val_loss: 1.1029 - val_accuracy: 0.5060
Epoch 6/100
193/193 [==============================] - 0s 316us/step - loss: 0.7505 - accuracy: 0.6736 - val_loss: 1.1066 - val_accuracy: 0.5301
Epoch 7/100
193/193 [==============================] - 0s 240us/step - loss: 0.7023 -

Epoch 57/100
193/193 [==============================] - 0s 252us/step - loss: 0.3393 - accuracy: 0.8601 - val_loss: 1.4577 - val_accuracy: 0.6024
Epoch 58/100
193/193 [==============================] - 0s 262us/step - loss: 0.3380 - accuracy: 0.8808 - val_loss: 1.4778 - val_accuracy: 0.5904
Epoch 59/100
193/193 [==============================] - 0s 199us/step - loss: 0.3242 - accuracy: 0.9016 - val_loss: 1.4979 - val_accuracy: 0.5904
Epoch 60/100
193/193 [==============================] - 0s 189us/step - loss: 0.3092 - accuracy: 0.9223 - val_loss: 1.5140 - val_accuracy: 0.5783
Epoch 61/100
193/193 [==============================] - 0s 187us/step - loss: 0.3119 - accuracy: 0.8912 - val_loss: 1.5612 - val_accuracy: 0.5663
Epoch 62/100
193/193 [==============================] - 0s 197us/step - loss: 0.3032 - accuracy: 0.8808 - val_loss: 1.5017 - val_accuracy: 0.5783
Epoch 63/100
193/193 [==============================] - 0s 195us/step - loss: 0.3185 - accuracy: 0.9119 - val_loss: 1.5397 -

In [21]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

83/83 [==============================] - 0s 133us/step
over-sampling test accuracy: 59.04%


In [22]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [23]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 193 samples, validate on 83 samples
Epoch 1/1000
193/193 [==============================] - 0s 1ms/step - loss: 1.0963 - accuracy: 0.3886 - val_loss: 0.9431 - val_accuracy: 0.5663
Epoch 2/1000
193/193 [==============================] - 0s 421us/step - loss: 0.9491 - accuracy: 0.5285 - val_loss: 0.8913 - val_accuracy: 0.5663
Epoch 3/1000
193/193 [==============================] - 0s 453us/step - loss: 0.8844 - accuracy: 0.5803 - val_loss: 0.9670 - val_accuracy: 0.5422
Epoch 4/1000
193/193 [==============================] - ETA: 0s - loss: 0.7844 - accuracy: 0.65 - 0s 568us/step - loss: 0.8451 - accuracy: 0.6425 - val_loss: 1.0888 - val_accuracy: 0.4940
Epoch 5/1000
193/193 [==============================] - 0s 587us/step - loss: 0.8315 - accuracy: 0.6321 - val_loss: 0.9461 - val_accuracy: 0.5783
Epoch 6/1000
193/193 [==============================] - 0s 484us/step - loss: 0.7371 - accuracy: 0.7047 - val_loss: 0.9541 - val_accuracy: 0.5542
Epoch 7/1000
193/193 [=================

193/193 [==============================] - 0s 316us/step - loss: 0.2371 - accuracy: 0.8964 - val_loss: 1.6357 - val_accuracy: 0.5422
Epoch 57/1000
193/193 [==============================] - 0s 341us/step - loss: 0.2314 - accuracy: 0.9223 - val_loss: 1.6149 - val_accuracy: 0.6145
Epoch 58/1000
193/193 [==============================] - 0s 325us/step - loss: 0.2236 - accuracy: 0.9275 - val_loss: 1.6419 - val_accuracy: 0.6265
Epoch 59/1000
193/193 [==============================] - 0s 329us/step - loss: 0.2509 - accuracy: 0.9171 - val_loss: 1.6627 - val_accuracy: 0.5422
Epoch 60/1000
193/193 [==============================] - 0s 461us/step - loss: 0.2347 - accuracy: 0.9378 - val_loss: 1.7079 - val_accuracy: 0.5783
Epoch 61/1000
193/193 [==============================] - 0s 323us/step - loss: 0.2238 - accuracy: 0.9275 - val_loss: 1.6638 - val_accuracy: 0.6265
Epoch 62/1000
193/193 [==============================] - 0s 329us/step - loss: 0.2206 - accuracy: 0.9223 - val_loss: 1.7116 - val_ac

Epoch 112/1000
193/193 [==============================] - 0s 410us/step - loss: 0.1403 - accuracy: 0.9482 - val_loss: 2.2270 - val_accuracy: 0.5904
Epoch 113/1000
193/193 [==============================] - 0s 319us/step - loss: 0.1424 - accuracy: 0.9534 - val_loss: 2.1646 - val_accuracy: 0.6024
Epoch 114/1000
193/193 [==============================] - 0s 390us/step - loss: 0.1454 - accuracy: 0.9430 - val_loss: 2.1107 - val_accuracy: 0.6024
Epoch 115/1000
193/193 [==============================] - 0s 374us/step - loss: 0.1383 - accuracy: 0.9482 - val_loss: 2.1927 - val_accuracy: 0.6024
Epoch 116/1000
193/193 [==============================] - 0s 303us/step - loss: 0.1207 - accuracy: 0.9637 - val_loss: 2.1808 - val_accuracy: 0.6024
Epoch 117/1000
193/193 [==============================] - 0s 298us/step - loss: 0.2393 - accuracy: 0.9275 - val_loss: 2.2156 - val_accuracy: 0.5663
Epoch 118/1000
193/193 [==============================] - 0s 296us/step - loss: 0.1646 - accuracy: 0.9430 - val_

193/193 [==============================] - 0s 312us/step - loss: 0.0982 - accuracy: 0.9689 - val_loss: 2.3162 - val_accuracy: 0.6145
Epoch 168/1000
193/193 [==============================] - 0s 328us/step - loss: 0.0991 - accuracy: 0.9585 - val_loss: 2.3351 - val_accuracy: 0.6024
Epoch 169/1000
193/193 [==============================] - 0s 380us/step - loss: 0.0928 - accuracy: 0.9741 - val_loss: 2.3834 - val_accuracy: 0.5783
Epoch 170/1000
193/193 [==============================] - 0s 312us/step - loss: 0.0887 - accuracy: 0.9637 - val_loss: 2.3675 - val_accuracy: 0.5783
Epoch 171/1000
193/193 [==============================] - 0s 299us/step - loss: 0.0881 - accuracy: 0.9689 - val_loss: 2.3976 - val_accuracy: 0.6024
Epoch 172/1000
193/193 [==============================] - 0s 406us/step - loss: 0.0958 - accuracy: 0.9637 - val_loss: 2.4124 - val_accuracy: 0.6024
Epoch 173/1000
193/193 [==============================] - 0s 350us/step - loss: 0.0924 - accuracy: 0.9637 - val_loss: 2.3884 - 

193/193 [==============================] - 0s 545us/step - loss: 0.0883 - accuracy: 0.9637 - val_loss: 2.6737 - val_accuracy: 0.5904
Epoch 223/1000
193/193 [==============================] - 0s 498us/step - loss: 0.0984 - accuracy: 0.9689 - val_loss: 2.6803 - val_accuracy: 0.5663
Epoch 224/1000
193/193 [==============================] - 0s 455us/step - loss: 0.0949 - accuracy: 0.9689 - val_loss: 2.7303 - val_accuracy: 0.5904
Epoch 225/1000
193/193 [==============================] - 0s 459us/step - loss: 0.0869 - accuracy: 0.9637 - val_loss: 2.6875 - val_accuracy: 0.5904
Epoch 226/1000
193/193 [==============================] - 0s 464us/step - loss: 0.0789 - accuracy: 0.9689 - val_loss: 2.6947 - val_accuracy: 0.5783
Epoch 227/1000
193/193 [==============================] - 0s 389us/step - loss: 0.0845 - accuracy: 0.9689 - val_loss: 2.7316 - val_accuracy: 0.6145
Epoch 228/1000
193/193 [==============================] - 0s 352us/step - loss: 0.0855 - accuracy: 0.9689 - val_loss: 2.6856 - 

...

193/193 [==============================] - 0s 953us/step - loss: 0.0587 - accuracy: 0.9741 - val_loss: 4.3308 - val_accuracy: 0.6024
Epoch 773/1000
193/193 [==============================] - 0s 535us/step - loss: 0.0568 - accuracy: 0.9689 - val_loss: 4.3279 - val_accuracy: 0.6024
Epoch 774/1000
193/193 [==============================] - 0s 509us/step - loss: 0.1932 - accuracy: 0.9378 - val_loss: 4.2425 - val_accuracy: 0.5904
Epoch 775/1000
193/193 [==============================] - 0s 480us/step - loss: 0.0582 - accuracy: 0.9741 - val_loss: 4.3797 - val_accuracy: 0.6024
Epoch 776/1000
193/193 [==============================] - 0s 527us/step - loss: 0.0681 - accuracy: 0.9741 - val_loss: 4.3722 - val_accuracy: 0.6024
Epoch 777/1000
193/193 [==============================] - 0s 609us/step - loss: 0.0589 - accuracy: 0.9637 - val_loss: 4.3315 - val_accuracy: 0.6024
Epoch 778/1000
193/193 [==============================] - 0s 552us/step - loss: 0.0554 - accuracy: 0.9689 - val_loss: 4.3257 - 

193/193 [==============================] - 0s 601us/step - loss: 0.0640 - accuracy: 0.9741 - val_loss: 3.4400 - val_accuracy: 0.6265
Epoch 828/1000
193/193 [==============================] - 0s 602us/step - loss: 0.0642 - accuracy: 0.9689 - val_loss: 3.4228 - val_accuracy: 0.6386
Epoch 829/1000
193/193 [==============================] - 0s 534us/step - loss: 0.0558 - accuracy: 0.9741 - val_loss: 3.4832 - val_accuracy: 0.6145
Epoch 830/1000
193/193 [==============================] - 0s 599us/step - loss: 0.0607 - accuracy: 0.9689 - val_loss: 3.5070 - val_accuracy: 0.6386
Epoch 831/1000
193/193 [==============================] - 0s 522us/step - loss: 0.0606 - accuracy: 0.9689 - val_loss: 3.4424 - val_accuracy: 0.6386
Epoch 832/1000
193/193 [==============================] - 0s 630us/step - loss: 0.0606 - accuracy: 0.9741 - val_loss: 3.4547 - val_accuracy: 0.6265
Epoch 833/1000
193/193 [==============================] - 0s 514us/step - loss: 0.0553 - accuracy: 0.9741 - val_loss: 3.5222 - 

193/193 [==============================] - 0s 330us/step - loss: 0.0548 - accuracy: 0.9689 - val_loss: 3.7127 - val_accuracy: 0.6506
Epoch 883/1000
193/193 [==============================] - 0s 362us/step - loss: 0.1176 - accuracy: 0.9585 - val_loss: 3.7729 - val_accuracy: 0.6386
Epoch 884/1000
193/193 [==============================] - 0s 352us/step - loss: 0.0663 - accuracy: 0.9585 - val_loss: 3.6668 - val_accuracy: 0.6627
Epoch 885/1000
193/193 [==============================] - 0s 341us/step - loss: 0.0688 - accuracy: 0.9482 - val_loss: 3.5126 - val_accuracy: 0.6747
Epoch 886/1000
193/193 [==============================] - 0s 351us/step - loss: 0.1192 - accuracy: 0.9430 - val_loss: 3.7408 - val_accuracy: 0.6627
Epoch 887/1000
193/193 [==============================] - 0s 326us/step - loss: 0.1642 - accuracy: 0.9326 - val_loss: 4.4916 - val_accuracy: 0.6145
Epoch 888/1000
193/193 [==============================] - 0s 411us/step - loss: 0.1488 - accuracy: 0.9430 - val_loss: 3.7764 - 

193/193 [==============================] - 0s 457us/step - loss: 0.0558 - accuracy: 0.9741 - val_loss: 4.7330 - val_accuracy: 0.6024
Epoch 938/1000
193/193 [==============================] - 0s 371us/step - loss: 0.0584 - accuracy: 0.9689 - val_loss: 4.7369 - val_accuracy: 0.6024
Epoch 939/1000
193/193 [==============================] - 0s 400us/step - loss: 0.0565 - accuracy: 0.9741 - val_loss: 4.7243 - val_accuracy: 0.6024
Epoch 940/1000
193/193 [==============================] - 0s 337us/step - loss: 0.0585 - accuracy: 0.9637 - val_loss: 4.7554 - val_accuracy: 0.6145
Epoch 941/1000
193/193 [==============================] - 0s 345us/step - loss: 0.0549 - accuracy: 0.9793 - val_loss: 4.7430 - val_accuracy: 0.6145
Epoch 942/1000
193/193 [==============================] - 0s 344us/step - loss: 0.0558 - accuracy: 0.9689 - val_loss: 4.7392 - val_accuracy: 0.6024
Epoch 943/1000
193/193 [==============================] - 0s 946us/step - loss: 0.0584 - accuracy: 0.9689 - val_loss: 4.7360 - 

193/193 [==============================] - 0s 346us/step - loss: 0.0527 - accuracy: 0.9741 - val_loss: 4.9993 - val_accuracy: 0.6145
Epoch 993/1000
193/193 [==============================] - 0s 336us/step - loss: 0.0540 - accuracy: 0.9689 - val_loss: 5.0056 - val_accuracy: 0.6145
Epoch 994/1000
193/193 [==============================] - 0s 318us/step - loss: 0.0551 - accuracy: 0.9689 - val_loss: 5.0052 - val_accuracy: 0.6145
Epoch 995/1000
193/193 [==============================] - 0s 353us/step - loss: 0.0548 - accuracy: 0.9741 - val_loss: 4.9835 - val_accuracy: 0.6145
Epoch 996/1000
193/193 [==============================] - 0s 355us/step - loss: 0.0538 - accuracy: 0.9741 - val_loss: 4.9903 - val_accuracy: 0.6145
Epoch 997/1000
193/193 [==============================] - 0s 724us/step - loss: 0.0517 - accuracy: 0.9741 - val_loss: 5.0180 - val_accuracy: 0.6145
Epoch 998/1000
193/193 [==============================] - 0s 924us/step - loss: 0.0540 - accuracy: 0.9637 - val_loss: 5.0236 - 

In [25]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

83/83 [==============================] - 0s 161us/step
over-sampling test accuracy: 61.45%


In [26]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [27]:
model1_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 46 samples, validate on 21 samples
Epoch 1/100
46/46 [==============================] - 0s 3ms/step - loss: 1.0809 - accuracy: 0.4130 - val_loss: 0.5588 - val_accuracy: 0.7143
Epoch 2/100
46/46 [==============================] - 0s 394us/step - loss: 0.5776 - accuracy: 0.8696 - val_loss: 0.4657 - val_accuracy: 0.8571
Epoch 3/100
46/46 [==============================] - 0s 601us/step - loss: 0.4403 - accuracy: 0.8913 - val_loss: 0.4163 - val_accuracy: 0.8571
Epoch 4/100
46/46 [==============================] - 0s 572us/step - loss: 0.3600 - accuracy: 0.9130 - val_loss: 0.3726 - val_accuracy: 0.8571
Epoch 5/100
46/46 [==============================] - 0s 431us/step - loss: 0.3090 - accuracy: 0.9130 - val_loss: 0.3337 - val_accuracy: 0.8571
Epoch 6/100
46/46 [==============================] - 0s 495us/step - loss: 0.2647 - accuracy: 0.9130 - val_loss: 0.3077 - val_accuracy: 0.8571
Epoch 7/100
46/46 [==============================] - 0s 485us/step - loss: 0.2229 - accuracy: 0.9130

Epoch 58/100
46/46 [==============================] - 0s 282us/step - loss: 0.0116 - accuracy: 1.0000 - val_loss: 0.2561 - val_accuracy: 0.8571
Epoch 59/100
46/46 [==============================] - 0s 309us/step - loss: 0.0115 - accuracy: 1.0000 - val_loss: 0.2454 - val_accuracy: 0.8571
Epoch 60/100
46/46 [==============================] - 0s 342us/step - loss: 0.0113 - accuracy: 1.0000 - val_loss: 0.2425 - val_accuracy: 0.8571
Epoch 61/100
46/46 [==============================] - 0s 389us/step - loss: 0.0110 - accuracy: 1.0000 - val_loss: 0.2465 - val_accuracy: 0.8571
Epoch 62/100
46/46 [==============================] - 0s 327us/step - loss: 0.0107 - accuracy: 1.0000 - val_loss: 0.2555 - val_accuracy: 0.8571
Epoch 63/100
46/46 [==============================] - 0s 376us/step - loss: 0.0103 - accuracy: 1.0000 - val_loss: 0.2679 - val_accuracy: 0.8571
Epoch 64/100
46/46 [==============================] - 0s 399us/step - loss: 0.0099 - accuracy: 1.0000 - val_loss: 0.2828 - val_accuracy:

In [29]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

21/21 [==============================] - 0s 115us/step
combination test accuracy: 85.71%


In [30]:
#### improve neural network on combination data
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [31]:
model1_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 46 samples, validate on 21 samples
Epoch 1/1000
46/46 [==============================] - 0s 7ms/step - loss: 0.8544 - accuracy: 0.4565 - val_loss: 0.7185 - val_accuracy: 0.4762
Epoch 2/1000
46/46 [==============================] - 0s 589us/step - loss: 0.6178 - accuracy: 0.7174 - val_loss: 0.6338 - val_accuracy: 0.9048
Epoch 3/1000
46/46 [==============================] - 0s 475us/step - loss: 0.5114 - accuracy: 0.9348 - val_loss: 0.5654 - val_accuracy: 0.8571
Epoch 4/1000
46/46 [==============================] - 0s 607us/step - loss: 0.4359 - accuracy: 0.8913 - val_loss: 0.5154 - val_accuracy: 0.8571
Epoch 5/1000
46/46 [==============================] - 0s 782us/step - loss: 0.3773 - accuracy: 0.8913 - val_loss: 0.4861 - val_accuracy: 0.8095
Epoch 6/1000
46/46 [==============================] - 0s 510us/step - loss: 0.3361 - accuracy: 0.9348 - val_loss: 0.4823 - val_accuracy: 0.7619
Epoch 7/1000
46/46 [==============================] - 0s 655us/step - loss: 0.2954 - accuracy:

46/46 [==============================] - 0s 765us/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 0.6386 - val_accuracy: 0.8571
Epoch 58/1000
46/46 [==============================] - 0s 849us/step - loss: 0.0078 - accuracy: 1.0000 - val_loss: 0.6349 - val_accuracy: 0.8571
Epoch 59/1000
46/46 [==============================] - 0s 694us/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.6323 - val_accuracy: 0.8571
Epoch 60/1000
46/46 [==============================] - 0s 793us/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.6269 - val_accuracy: 0.8571
Epoch 61/1000
46/46 [==============================] - 0s 529us/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 0.6224 - val_accuracy: 0.8571
Epoch 62/1000
46/46 [==============================] - 0s 655us/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 0.6220 - val_accuracy: 0.8571
Epoch 63/1000
46/46 [==============================] - 0s 496us/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 0.6313 - val_accuracy: 0.8571

46/46 [==============================] - 0s 351us/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.8221 - val_accuracy: 0.8571
Epoch 114/1000
46/46 [==============================] - 0s 343us/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.8221 - val_accuracy: 0.8571
Epoch 115/1000
46/46 [==============================] - 0s 563us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.8189 - val_accuracy: 0.8571
Epoch 116/1000
46/46 [==============================] - 0s 510us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.8161 - val_accuracy: 0.8571
Epoch 117/1000
46/46 [==============================] - 0s 536us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.8129 - val_accuracy: 0.8571
Epoch 118/1000
46/46 [==============================] - 0s 889us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.8125 - val_accuracy: 0.8571
Epoch 119/1000
46/46 [==============================] - 0s 612us/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.8126 - val_accuracy: 

46/46 [==============================] - 0s 652us/step - loss: 6.3357e-04 - accuracy: 1.0000 - val_loss: 0.9135 - val_accuracy: 0.8571
Epoch 169/1000
46/46 [==============================] - 0s 351us/step - loss: 6.2724e-04 - accuracy: 1.0000 - val_loss: 0.9079 - val_accuracy: 0.8571
Epoch 170/1000
46/46 [==============================] - 0s 578us/step - loss: 6.2002e-04 - accuracy: 1.0000 - val_loss: 0.9041 - val_accuracy: 0.8571
Epoch 171/1000
46/46 [==============================] - 0s 707us/step - loss: 6.1147e-04 - accuracy: 1.0000 - val_loss: 0.9045 - val_accuracy: 0.8571
Epoch 172/1000
46/46 [==============================] - 0s 752us/step - loss: 6.0845e-04 - accuracy: 1.0000 - val_loss: 0.9076 - val_accuracy: 0.8571
Epoch 173/1000
46/46 [==============================] - 0s 509us/step - loss: 5.9709e-04 - accuracy: 1.0000 - val_loss: 0.9133 - val_accuracy: 0.8571
Epoch 174/1000
46/46 [==============================] - 0s 468us/step - loss: 5.8895e-04 - accuracy: 1.0000 - val_l

Epoch 223/1000
46/46 [==============================] - 0s 350us/step - loss: 3.5069e-04 - accuracy: 1.0000 - val_loss: 0.9827 - val_accuracy: 0.8571
Epoch 224/1000
46/46 [==============================] - 0s 383us/step - loss: 3.4726e-04 - accuracy: 1.0000 - val_loss: 0.9849 - val_accuracy: 0.8571
Epoch 225/1000
46/46 [==============================] - 0s 380us/step - loss: 3.4438e-04 - accuracy: 1.0000 - val_loss: 0.9871 - val_accuracy: 0.8571
Epoch 226/1000
46/46 [==============================] - 0s 442us/step - loss: 3.4150e-04 - accuracy: 1.0000 - val_loss: 0.9870 - val_accuracy: 0.8571
Epoch 227/1000
46/46 [==============================] - 0s 401us/step - loss: 3.3840e-04 - accuracy: 1.0000 - val_loss: 0.9874 - val_accuracy: 0.8571
Epoch 228/1000
46/46 [==============================] - 0s 430us/step - loss: 3.3476e-04 - accuracy: 1.0000 - val_loss: 0.9865 - val_accuracy: 0.8571
Epoch 229/1000
46/46 [==============================] - 0s 420us/step - loss: 3.3214e-04 - accuracy:

...

46/46 [==============================] - 0s 423us/step - loss: 2.4827e-05 - accuracy: 1.0000 - val_loss: 1.2971 - val_accuracy: 0.8571
Epoch 768/1000
46/46 [==============================] - 0s 352us/step - loss: 2.4760e-05 - accuracy: 1.0000 - val_loss: 1.2977 - val_accuracy: 0.8571
Epoch 769/1000
46/46 [==============================] - 0s 339us/step - loss: 2.4690e-05 - accuracy: 1.0000 - val_loss: 1.2982 - val_accuracy: 0.8571
Epoch 770/1000
46/46 [==============================] - 0s 378us/step - loss: 2.4633e-05 - accuracy: 1.0000 - val_loss: 1.2991 - val_accuracy: 0.8571
Epoch 771/1000
46/46 [==============================] - 0s 413us/step - loss: 2.4542e-05 - accuracy: 1.0000 - val_loss: 1.2991 - val_accuracy: 0.8571
Epoch 772/1000
46/46 [==============================] - 0s 332us/step - loss: 2.4470e-05 - accuracy: 1.0000 - val_loss: 1.2997 - val_accuracy: 0.8571
Epoch 773/1000
46/46 [==============================] - 0s 413us/step - loss: 2.4392e-05 - accuracy: 1.0000 - val_l

Epoch 822/1000
46/46 [==============================] - 0s 484us/step - loss: 2.1246e-05 - accuracy: 1.0000 - val_loss: 1.3107 - val_accuracy: 0.8571
Epoch 823/1000
46/46 [==============================] - 0s 328us/step - loss: 2.1182e-05 - accuracy: 1.0000 - val_loss: 1.3109 - val_accuracy: 0.8571
Epoch 824/1000
46/46 [==============================] - 0s 379us/step - loss: 2.1135e-05 - accuracy: 1.0000 - val_loss: 1.3113 - val_accuracy: 0.8571
Epoch 825/1000
46/46 [==============================] - 0s 410us/step - loss: 2.1042e-05 - accuracy: 1.0000 - val_loss: 1.3120 - val_accuracy: 0.8571
Epoch 826/1000
46/46 [==============================] - 0s 336us/step - loss: 2.0982e-05 - accuracy: 1.0000 - val_loss: 1.3128 - val_accuracy: 0.8571
Epoch 827/1000
46/46 [==============================] - 0s 381us/step - loss: 2.0928e-05 - accuracy: 1.0000 - val_loss: 1.3140 - val_accuracy: 0.8571
Epoch 828/1000
46/46 [==============================] - 0s 337us/step - loss: 2.0860e-05 - accuracy:

46/46 [==============================] - 0s 456us/step - loss: 1.8287e-05 - accuracy: 1.0000 - val_loss: 1.3338 - val_accuracy: 0.8571
Epoch 877/1000
46/46 [==============================] - 0s 362us/step - loss: 1.8235e-05 - accuracy: 1.0000 - val_loss: 1.3332 - val_accuracy: 0.8571
Epoch 878/1000
46/46 [==============================] - 0s 337us/step - loss: 1.8189e-05 - accuracy: 1.0000 - val_loss: 1.3330 - val_accuracy: 0.8571
Epoch 879/1000
46/46 [==============================] - 0s 405us/step - loss: 1.8140e-05 - accuracy: 1.0000 - val_loss: 1.3335 - val_accuracy: 0.8571
Epoch 880/1000
46/46 [==============================] - 0s 443us/step - loss: 1.8096e-05 - accuracy: 1.0000 - val_loss: 1.3341 - val_accuracy: 0.8571
Epoch 881/1000
46/46 [==============================] - 0s 437us/step - loss: 1.8054e-05 - accuracy: 1.0000 - val_loss: 1.3347 - val_accuracy: 0.8571
Epoch 882/1000
46/46 [==============================] - 0s 383us/step - loss: 1.7984e-05 - accuracy: 1.0000 - val_l

Epoch 931/1000
46/46 [==============================] - 0s 312us/step - loss: 1.5976e-05 - accuracy: 1.0000 - val_loss: 1.3517 - val_accuracy: 0.8571
Epoch 932/1000
46/46 [==============================] - 0s 286us/step - loss: 1.5942e-05 - accuracy: 1.0000 - val_loss: 1.3524 - val_accuracy: 0.8571
Epoch 933/1000
46/46 [==============================] - 0s 309us/step - loss: 1.5901e-05 - accuracy: 1.0000 - val_loss: 1.3526 - val_accuracy: 0.8571
Epoch 934/1000
46/46 [==============================] - 0s 361us/step - loss: 1.5880e-05 - accuracy: 1.0000 - val_loss: 1.3531 - val_accuracy: 0.8571
Epoch 935/1000
46/46 [==============================] - 0s 334us/step - loss: 1.5839e-05 - accuracy: 1.0000 - val_loss: 1.3537 - val_accuracy: 0.8571
Epoch 936/1000
46/46 [==============================] - 0s 349us/step - loss: 1.5807e-05 - accuracy: 1.0000 - val_loss: 1.3543 - val_accuracy: 0.8571
Epoch 937/1000
46/46 [==============================] - 0s 313us/step - loss: 1.5776e-05 - accuracy:

46/46 [==============================] - 0s 386us/step - loss: 1.4048e-05 - accuracy: 1.0000 - val_loss: 1.3677 - val_accuracy: 0.8571
Epoch 986/1000
46/46 [==============================] - 0s 432us/step - loss: 1.4004e-05 - accuracy: 1.0000 - val_loss: 1.3680 - val_accuracy: 0.8571
Epoch 987/1000
46/46 [==============================] - 0s 326us/step - loss: 1.3962e-05 - accuracy: 1.0000 - val_loss: 1.3683 - val_accuracy: 0.8571
Epoch 988/1000
46/46 [==============================] - 0s 424us/step - loss: 1.3934e-05 - accuracy: 1.0000 - val_loss: 1.3685 - val_accuracy: 0.8571
Epoch 989/1000
46/46 [==============================] - 0s 409us/step - loss: 1.3892e-05 - accuracy: 1.0000 - val_loss: 1.3685 - val_accuracy: 0.8571
Epoch 990/1000
46/46 [==============================] - 0s 427us/step - loss: 1.3864e-05 - accuracy: 1.0000 - val_loss: 1.3686 - val_accuracy: 0.8571
Epoch 991/1000
46/46 [==============================] - 0s 378us/step - loss: 1.3833e-05 - accuracy: 1.0000 - val_l

In [33]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

21/21 [==============================] - 0s 294us/step
combination test accuracy: 85.71%


In [34]:
############ Feature selection using lasso ##########

In [92]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [93]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [94]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [95]:
X_train_features_comb = np.vstack((names, X_train_comb))
X_train_features_comb = pd.DataFrame(X_train_features_comb)

In [96]:
sel_features = X_train_features_comb.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_comb.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 7002
selected features: 120


In [97]:
cols = sel_features.values
cols.reshape((1, -1))

array([[ 148,  342,  403,  425,  462,  488,  534,  570,  728,  802,  908,
         952,  955,  991,  994, 1016, 1100, 1154, 1249, 1283, 1290, 1479,
        1536, 1606, 1663, 1671, 1672, 1681, 1799, 1925, 2106, 2250, 2283,
        2284, 2318, 2319, 2326, 2587, 2594, 2614, 2618, 2643, 2759, 2791,
        2815, 2830, 2871, 3102, 3171, 3173, 3206, 3354, 3359, 3507, 3817,
        3825, 3836, 3837, 3965, 4027, 4031, 4045, 4046, 4047, 4059, 4331,
        4336, 4344, 4351, 4455, 4474, 4505, 4552, 4586, 4736, 4842, 4880,
        4939, 5012, 5047, 5051, 5117, 5131, 5339, 5363, 5382, 5408, 5428,
        5454, 5455, 5468, 5469, 5492, 5535, 5591, 5689, 5690, 5707, 5771,
        5831, 5913, 5972, 6019, 6024, 6187, 6231, 6251, 6303, 6393, 6404,
        6423, 6460, 6547, 6609, 6642, 6646, 6691, 6694, 6755, 6935]])

In [98]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTGGTGTTCGGAGATAAATATATCGGCGCTTCTCTACTTTCAATATTAGCTGAAGCATATGAATCGATAATAATTCGATTGTAGTCAGATGTTAAAGC',
       'TTTGGTGTTCGGAGATAAATATATCGGCGCTTCTCTACTTTCAATATTAGCTGAAGCATATGAATCGATAATAATTCGATTGTAGTCAGATGTTAAAGCG',
       'TTTGAATGCTAACGTAACTATTTCCATCTCTTCGTTTAACTAAGTTGAATTTGCCCATACCTGATTCAATTGTAGTGTCTCCGCCTGTAGCATAGTCACC',
       'TTTCTTGATTTTTA',
       'TTTCGTATGCACTATCAGCGTTTGATAATGTGAATGCGAATCGGTTTAAACCAGGTTTATTTTTGGTGTTCGGAGATAAATATATCGGCGCTTCTCTACT',
       'TTTCCATTTATTTC',
       'TTTATTTTTGGTGTTCGGAGATAAATATATCGGCGCTTCTCTACTTTCAATATTAGCTGAAGCATATGAATCGATAATAATTCGATTGTAGTCAGATGTT',
       'TTTATCTATATCA',
       'TTTAAACCAGGTTTATTTTTGGTGTTCGGAGATAAATATATCGGCGCTTCTCTACTTTCAATATTAGCTGAAGCATATGAATCGATAATAATTCGATTGT',
       'TTGTATATAAATA', 'TTGCCATTCACTCATTAACTCGT', 'TTGATAGTAAAGG',
       'TTGATAATGTGAATGCGAATCGGTTTAAACCAGGTTTATTTTTGGTGTTCGGAGATAAATATATCGGCGCTTCTCTACTTTCAATATTAGCTGAAGCATA',
       'TTGAATGCTAACGTAACTATTTCCATCTCTTCGTTTAACTAAGTTGAATTTGCCCATACCTGATTCAATTGTA

In [99]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 120) (255,) (255, 121)


In [100]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 92), (1, 92), (2, 92)]


In [101]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_sel_comb, y_sel_comb = smote_enn.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_comb).items()))

[(0, 20), (1, 18), (2, 35)]


In [102]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_over)

In [103]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_sel_train_comb, X_sel_test_comb, y_sel_train_comb, y_sel_test_comb = train_test_split(X_sel_comb, y_sel_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_comb)

In [104]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [105]:
model2_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [106]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 193 samples, validate on 83 samples
Epoch 1/100
193/193 [==============================] - 0s 663us/step - loss: 1.1612 - accuracy: 0.4301 - val_loss: 1.0723 - val_accuracy: 0.4458
Epoch 2/100
193/193 [==============================] - 0s 103us/step - loss: 1.0008 - accuracy: 0.5233 - val_loss: 1.0255 - val_accuracy: 0.5301
Epoch 3/100
193/193 [==============================] - 0s 125us/step - loss: 0.9656 - accuracy: 0.6010 - val_loss: 1.0030 - val_accuracy: 0.5301
Epoch 4/100
193/193 [==============================] - 0s 139us/step - loss: 0.9512 - accuracy: 0.6114 - val_loss: 0.9972 - val_accuracy: 0.5301
Epoch 5/100
193/193 [==============================] - 0s 112us/step - loss: 0.9364 - accuracy: 0.6218 - val_loss: 0.9873 - val_accuracy: 0.5422
Epoch 6/100
193/193 [==============================] - 0s 113us/step - loss: 0.9264 - accuracy: 0.6010 - val_loss: 0.9862 - val_accuracy: 0.5542
Epoch 7/100
193/193 [==============================] - 0s 112us/step - loss: 0.9181 -

193/193 [==============================] - 0s 158us/step - loss: 0.7324 - accuracy: 0.6995 - val_loss: 0.8696 - val_accuracy: 0.5783
Epoch 57/100
193/193 [==============================] - 0s 190us/step - loss: 0.7305 - accuracy: 0.7098 - val_loss: 0.8642 - val_accuracy: 0.5663
Epoch 58/100
193/193 [==============================] - 0s 127us/step - loss: 0.7389 - accuracy: 0.7047 - val_loss: 0.8827 - val_accuracy: 0.5663
Epoch 59/100
193/193 [==============================] - 0s 130us/step - loss: 0.7423 - accuracy: 0.6891 - val_loss: 0.8925 - val_accuracy: 0.5663
Epoch 60/100
193/193 [==============================] - 0s 118us/step - loss: 0.7421 - accuracy: 0.6839 - val_loss: 0.8970 - val_accuracy: 0.5783
Epoch 61/100
193/193 [==============================] - 0s 120us/step - loss: 0.7378 - accuracy: 0.6891 - val_loss: 0.8850 - val_accuracy: 0.5904
Epoch 62/100
193/193 [==============================] - 0s 156us/step - loss: 0.7278 - accuracy: 0.6943 - val_loss: 0.8838 - val_accuracy

In [107]:
acc_test_over2 = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over2*100))

83/83 [==============================] - 0s 110us/step
combination test accuracy: 56.63%


In [112]:
#### improve neural network on over-sampling data
model2_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [113]:
model2_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [114]:
model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 193 samples, validate on 83 samples
Epoch 1/1000
193/193 [==============================] - 0s 691us/step - loss: 1.0463 - accuracy: 0.4456 - val_loss: 0.9845 - val_accuracy: 0.4699
Epoch 2/1000
193/193 [==============================] - 0s 104us/step - loss: 1.0090 - accuracy: 0.5078 - val_loss: 0.9711 - val_accuracy: 0.5060
Epoch 3/1000
193/193 [==============================] - 0s 163us/step - loss: 0.9888 - accuracy: 0.5337 - val_loss: 0.9634 - val_accuracy: 0.4699
Epoch 4/1000
193/193 [==============================] - 0s 124us/step - loss: 0.9794 - accuracy: 0.5544 - val_loss: 0.9593 - val_accuracy: 0.5181
Epoch 5/1000
193/193 [==============================] - 0s 121us/step - loss: 0.9671 - accuracy: 0.5855 - val_loss: 0.9507 - val_accuracy: 0.5422
Epoch 6/1000
193/193 [==============================] - 0s 125us/step - loss: 0.9553 - accuracy: 0.6062 - val_loss: 0.9405 - val_accuracy: 0.5542
Epoch 7/1000
193/193 [==============================] - 0s 125us/step - loss: 0

193/193 [==============================] - 0s 130us/step - loss: 0.6946 - accuracy: 0.7150 - val_loss: 0.8540 - val_accuracy: 0.6145
Epoch 57/1000
193/193 [==============================] - 0s 134us/step - loss: 0.6864 - accuracy: 0.7202 - val_loss: 0.8553 - val_accuracy: 0.6024
Epoch 58/1000
193/193 [==============================] - 0s 113us/step - loss: 0.6800 - accuracy: 0.7202 - val_loss: 0.8626 - val_accuracy: 0.6024
Epoch 59/1000
193/193 [==============================] - 0s 91us/step - loss: 0.6705 - accuracy: 0.7358 - val_loss: 0.8631 - val_accuracy: 0.6024
Epoch 60/1000
193/193 [==============================] - 0s 125us/step - loss: 0.6641 - accuracy: 0.7254 - val_loss: 0.8609 - val_accuracy: 0.6024
Epoch 61/1000
193/193 [==============================] - 0s 102us/step - loss: 0.6705 - accuracy: 0.7047 - val_loss: 0.8556 - val_accuracy: 0.6024
Epoch 62/1000
193/193 [==============================] - 0s 105us/step - loss: 0.6630 - accuracy: 0.7202 - val_loss: 0.8624 - val_acc

Epoch 112/1000
193/193 [==============================] - 0s 131us/step - loss: 0.5536 - accuracy: 0.7565 - val_loss: 0.9375 - val_accuracy: 0.6145
Epoch 113/1000
193/193 [==============================] - 0s 118us/step - loss: 0.5569 - accuracy: 0.7720 - val_loss: 0.9399 - val_accuracy: 0.5904
Epoch 114/1000
193/193 [==============================] - 0s 102us/step - loss: 0.5456 - accuracy: 0.7668 - val_loss: 0.9392 - val_accuracy: 0.6024
Epoch 115/1000
193/193 [==============================] - 0s 118us/step - loss: 0.5401 - accuracy: 0.7565 - val_loss: 0.9415 - val_accuracy: 0.5783
Epoch 116/1000
193/193 [==============================] - 0s 131us/step - loss: 0.5383 - accuracy: 0.7565 - val_loss: 0.9443 - val_accuracy: 0.6024
Epoch 117/1000
193/193 [==============================] - 0s 116us/step - loss: 0.5595 - accuracy: 0.7565 - val_loss: 0.9820 - val_accuracy: 0.6024
Epoch 118/1000
193/193 [==============================] - 0s 112us/step - loss: 0.6078 - accuracy: 0.7461 - val_

193/193 [==============================] - 0s 105us/step - loss: 0.4700 - accuracy: 0.7876 - val_loss: 0.9856 - val_accuracy: 0.6024
Epoch 168/1000
193/193 [==============================] - 0s 106us/step - loss: 0.4915 - accuracy: 0.7824 - val_loss: 1.0471 - val_accuracy: 0.6145
Epoch 169/1000
193/193 [==============================] - 0s 95us/step - loss: 0.5051 - accuracy: 0.7824 - val_loss: 1.0110 - val_accuracy: 0.5783
Epoch 170/1000
193/193 [==============================] - 0s 100us/step - loss: 0.4776 - accuracy: 0.7824 - val_loss: 0.9965 - val_accuracy: 0.5663
Epoch 171/1000
193/193 [==============================] - 0s 98us/step - loss: 0.4764 - accuracy: 0.7876 - val_loss: 1.0090 - val_accuracy: 0.5542
Epoch 172/1000
193/193 [==============================] - 0s 101us/step - loss: 0.4941 - accuracy: 0.8031 - val_loss: 1.0180 - val_accuracy: 0.5663
Epoch 173/1000
193/193 [==============================] - 0s 113us/step - loss: 0.4821 - accuracy: 0.8083 - val_loss: 1.0406 - va

193/193 [==============================] - 0s 100us/step - loss: 0.4332 - accuracy: 0.8031 - val_loss: 1.0416 - val_accuracy: 0.6024
Epoch 223/1000
193/193 [==============================] - 0s 119us/step - loss: 0.4358 - accuracy: 0.8290 - val_loss: 1.0546 - val_accuracy: 0.5663
Epoch 224/1000
193/193 [==============================] - 0s 107us/step - loss: 0.4476 - accuracy: 0.8135 - val_loss: 1.0712 - val_accuracy: 0.5663
Epoch 225/1000
193/193 [==============================] - 0s 94us/step - loss: 0.4455 - accuracy: 0.8135 - val_loss: 1.0765 - val_accuracy: 0.5542
Epoch 226/1000
193/193 [==============================] - 0s 105us/step - loss: 0.4368 - accuracy: 0.7979 - val_loss: 1.0866 - val_accuracy: 0.5422
Epoch 227/1000
193/193 [==============================] - 0s 96us/step - loss: 0.4451 - accuracy: 0.7979 - val_loss: 1.1218 - val_accuracy: 0.5904
Epoch 228/1000
193/193 [==============================] - 0s 105us/step - loss: 0.4412 - accuracy: 0.7876 - val_loss: 1.0859 - va

...

193/193 [==============================] - 0s 102us/step - loss: 0.3302 - accuracy: 0.8601 - val_loss: 1.7415 - val_accuracy: 0.5663
Epoch 773/1000
193/193 [==============================] - 0s 99us/step - loss: 0.3107 - accuracy: 0.8705 - val_loss: 1.7393 - val_accuracy: 0.5904
Epoch 774/1000
193/193 [==============================] - 0s 115us/step - loss: 0.3070 - accuracy: 0.8653 - val_loss: 1.6924 - val_accuracy: 0.5783
Epoch 775/1000
193/193 [==============================] - 0s 104us/step - loss: 0.3095 - accuracy: 0.8653 - val_loss: 1.6571 - val_accuracy: 0.6024
Epoch 776/1000
193/193 [==============================] - 0s 97us/step - loss: 0.3446 - accuracy: 0.8653 - val_loss: 1.6552 - val_accuracy: 0.6145
Epoch 777/1000
193/193 [==============================] - 0s 108us/step - loss: 0.3340 - accuracy: 0.8653 - val_loss: 1.6426 - val_accuracy: 0.6145
Epoch 778/1000
193/193 [==============================] - 0s 97us/step - loss: 0.3201 - accuracy: 0.8601 - val_loss: 1.6502 - val

193/193 [==============================] - 0s 112us/step - loss: 0.3281 - accuracy: 0.8705 - val_loss: 1.9204 - val_accuracy: 0.5783
Epoch 828/1000
193/193 [==============================] - 0s 116us/step - loss: 0.3362 - accuracy: 0.8705 - val_loss: 1.8486 - val_accuracy: 0.5663
Epoch 829/1000
193/193 [==============================] - 0s 137us/step - loss: 0.2938 - accuracy: 0.8653 - val_loss: 1.8261 - val_accuracy: 0.5663
Epoch 830/1000
193/193 [==============================] - 0s 131us/step - loss: 0.3223 - accuracy: 0.8290 - val_loss: 2.0239 - val_accuracy: 0.5783
Epoch 831/1000
193/193 [==============================] - 0s 99us/step - loss: 0.3617 - accuracy: 0.8238 - val_loss: 1.9271 - val_accuracy: 0.5783
Epoch 832/1000
193/193 [==============================] - 0s 102us/step - loss: 0.3219 - accuracy: 0.8446 - val_loss: 1.7781 - val_accuracy: 0.5663
Epoch 833/1000
193/193 [==============================] - 0s 128us/step - loss: 0.3344 - accuracy: 0.8808 - val_loss: 1.8437 - v

193/193 [==============================] - 0s 107us/step - loss: 0.2924 - accuracy: 0.8601 - val_loss: 1.7533 - val_accuracy: 0.6024
Epoch 883/1000
193/193 [==============================] - 0s 102us/step - loss: 0.2935 - accuracy: 0.8549 - val_loss: 1.7284 - val_accuracy: 0.5904
Epoch 884/1000
193/193 [==============================] - 0s 101us/step - loss: 0.2907 - accuracy: 0.8705 - val_loss: 1.7129 - val_accuracy: 0.5904
Epoch 885/1000
193/193 [==============================] - 0s 141us/step - loss: 0.2877 - accuracy: 0.8705 - val_loss: 1.7096 - val_accuracy: 0.5904
Epoch 886/1000
193/193 [==============================] - 0s 156us/step - loss: 0.2887 - accuracy: 0.8653 - val_loss: 1.7187 - val_accuracy: 0.5904
Epoch 887/1000
193/193 [==============================] - 0s 102us/step - loss: 0.3024 - accuracy: 0.8497 - val_loss: 1.7745 - val_accuracy: 0.6024
Epoch 888/1000
193/193 [==============================] - 0s 102us/step - loss: 0.3396 - accuracy: 0.8549 - val_loss: 1.7806 - 

193/193 [==============================] - 0s 107us/step - loss: 0.2803 - accuracy: 0.8705 - val_loss: 1.9076 - val_accuracy: 0.5663
Epoch 938/1000
193/193 [==============================] - 0s 104us/step - loss: 0.2772 - accuracy: 0.8756 - val_loss: 1.8789 - val_accuracy: 0.5904
Epoch 939/1000
193/193 [==============================] - 0s 97us/step - loss: 0.2749 - accuracy: 0.8756 - val_loss: 1.8803 - val_accuracy: 0.6024
Epoch 940/1000
193/193 [==============================] - 0s 98us/step - loss: 0.2753 - accuracy: 0.8756 - val_loss: 1.8776 - val_accuracy: 0.5783
Epoch 941/1000
193/193 [==============================] - 0s 100us/step - loss: 0.2804 - accuracy: 0.8756 - val_loss: 1.8687 - val_accuracy: 0.6024
Epoch 942/1000
193/193 [==============================] - 0s 105us/step - loss: 0.2984 - accuracy: 0.8705 - val_loss: 1.8728 - val_accuracy: 0.6024
Epoch 943/1000
193/193 [==============================] - 0s 102us/step - loss: 0.2886 - accuracy: 0.8756 - val_loss: 1.8912 - va

193/193 [==============================] - 0s 118us/step - loss: 0.2845 - accuracy: 0.8756 - val_loss: 1.8881 - val_accuracy: 0.5904
Epoch 993/1000
193/193 [==============================] - 0s 110us/step - loss: 0.2829 - accuracy: 0.8705 - val_loss: 1.9288 - val_accuracy: 0.5783
Epoch 994/1000
193/193 [==============================] - 0s 101us/step - loss: 0.2757 - accuracy: 0.8705 - val_loss: 1.9533 - val_accuracy: 0.5663
Epoch 995/1000
193/193 [==============================] - 0s 105us/step - loss: 0.2739 - accuracy: 0.8756 - val_loss: 1.9358 - val_accuracy: 0.5663
Epoch 996/1000
193/193 [==============================] - 0s 141us/step - loss: 0.2725 - accuracy: 0.8756 - val_loss: 1.9284 - val_accuracy: 0.5783
Epoch 997/1000
193/193 [==============================] - 0s 102us/step - loss: 0.2738 - accuracy: 0.8756 - val_loss: 1.9273 - val_accuracy: 0.5783
Epoch 998/1000
193/193 [==============================] - 0s 105us/step - loss: 0.2714 - accuracy: 0.8705 - val_loss: 1.9266 - 

In [115]:
acc_test_over3 = model2_over2.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over3*100))

83/83 [==============================] - 0s 102us/step
combination test accuracy: 59.04%


In [116]:
#### neural network on combination data
model2_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [117]:
model2_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [118]:
model2_comb.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 51 samples, validate on 22 samples
Epoch 1/100
51/51 [==============================] - 0s 2ms/step - loss: 1.7018 - accuracy: 0.0392 - val_loss: 1.5241 - val_accuracy: 0.0909
Epoch 2/100
51/51 [==============================] - 0s 196us/step - loss: 1.3662 - accuracy: 0.0196 - val_loss: 1.2689 - val_accuracy: 0.2273
Epoch 3/100
51/51 [==============================] - 0s 257us/step - loss: 1.0652 - accuracy: 0.3725 - val_loss: 1.0828 - val_accuracy: 0.6364
Epoch 4/100
51/51 [==============================] - 0s 217us/step - loss: 0.8858 - accuracy: 0.5686 - val_loss: 0.9570 - val_accuracy: 0.6364
Epoch 5/100
51/51 [==============================] - 0s 208us/step - loss: 0.7562 - accuracy: 0.5882 - val_loss: 0.8743 - val_accuracy: 0.6364
Epoch 6/100
51/51 [==============================] - 0s 232us/step - loss: 0.6903 - accuracy: 0.6078 - val_loss: 0.8149 - val_accuracy: 0.6364
Epoch 7/100
51/51 [==============================] - 0s 227us/step - loss: 0.6492 - accuracy: 0.6863

Epoch 58/100
51/51 [==============================] - 0s 435us/step - loss: 0.1505 - accuracy: 0.9804 - val_loss: 0.2775 - val_accuracy: 0.9091
Epoch 59/100
51/51 [==============================] - 0s 314us/step - loss: 0.1467 - accuracy: 0.9804 - val_loss: 0.2753 - val_accuracy: 0.9091
Epoch 60/100
51/51 [==============================] - 0s 328us/step - loss: 0.1436 - accuracy: 0.9804 - val_loss: 0.2742 - val_accuracy: 0.8636
Epoch 61/100
51/51 [==============================] - 0s 381us/step - loss: 0.1403 - accuracy: 0.9804 - val_loss: 0.2727 - val_accuracy: 0.8636
Epoch 62/100
51/51 [==============================] - 0s 182us/step - loss: 0.1375 - accuracy: 0.9804 - val_loss: 0.2709 - val_accuracy: 0.8636
Epoch 63/100
51/51 [==============================] - 0s 239us/step - loss: 0.1347 - accuracy: 0.9804 - val_loss: 0.2684 - val_accuracy: 0.8636
Epoch 64/100
51/51 [==============================] - 0s 277us/step - loss: 0.1319 - accuracy: 0.9804 - val_loss: 0.2690 - val_accuracy:

In [119]:
acc_test_comb2 = model2_comb.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

22/22 [==============================] - 0s 140us/step
combination test accuracy: 81.82%


In [123]:
#### improve neural network
model2_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    
])

In [124]:
model2_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [125]:
model2_comb2.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 51 samples, validate on 22 samples
Epoch 1/1000
51/51 [==============================] - 0s 3ms/step - loss: 1.2018 - accuracy: 0.1569 - val_loss: 1.0945 - val_accuracy: 0.3636
Epoch 2/1000
51/51 [==============================] - 0s 167us/step - loss: 1.0303 - accuracy: 0.5098 - val_loss: 0.9890 - val_accuracy: 0.3636
Epoch 3/1000
51/51 [==============================] - 0s 197us/step - loss: 0.9104 - accuracy: 0.5098 - val_loss: 0.9098 - val_accuracy: 0.4545
Epoch 4/1000
51/51 [==============================] - 0s 303us/step - loss: 0.8315 - accuracy: 0.5098 - val_loss: 0.8471 - val_accuracy: 0.5000
Epoch 5/1000
51/51 [==============================] - 0s 308us/step - loss: 0.7709 - accuracy: 0.5098 - val_loss: 0.7977 - val_accuracy: 0.5000
Epoch 6/1000
51/51 [==============================] - 0s 340us/step - loss: 0.7268 - accuracy: 0.5098 - val_loss: 0.7573 - val_accuracy: 0.5000
Epoch 7/1000
51/51 [==============================] - 0s 356us/step - loss: 0.6916 - accuracy:

Epoch 57/1000
51/51 [==============================] - 0s 230us/step - loss: 0.0689 - accuracy: 0.9804 - val_loss: 0.1259 - val_accuracy: 1.0000
Epoch 58/1000
51/51 [==============================] - 0s 197us/step - loss: 0.0668 - accuracy: 0.9804 - val_loss: 0.1245 - val_accuracy: 1.0000
Epoch 59/1000
51/51 [==============================] - 0s 219us/step - loss: 0.0656 - accuracy: 0.9804 - val_loss: 0.1206 - val_accuracy: 1.0000
Epoch 60/1000
51/51 [==============================] - 0s 269us/step - loss: 0.0634 - accuracy: 0.9804 - val_loss: 0.1219 - val_accuracy: 1.0000
Epoch 61/1000
51/51 [==============================] - 0s 246us/step - loss: 0.0616 - accuracy: 0.9804 - val_loss: 0.1207 - val_accuracy: 1.0000
Epoch 62/1000
51/51 [==============================] - 0s 180us/step - loss: 0.0602 - accuracy: 0.9804 - val_loss: 0.1195 - val_accuracy: 1.0000
Epoch 63/1000
51/51 [==============================] - 0s 194us/step - loss: 0.0590 - accuracy: 0.9804 - val_loss: 0.1211 - val_ac

51/51 [==============================] - 0s 158us/step - loss: 0.0311 - accuracy: 0.9804 - val_loss: 0.1012 - val_accuracy: 0.9545
Epoch 114/1000
51/51 [==============================] - 0s 188us/step - loss: 0.0313 - accuracy: 0.9804 - val_loss: 0.1037 - val_accuracy: 0.9545
Epoch 115/1000
51/51 [==============================] - 0s 203us/step - loss: 0.0300 - accuracy: 0.9804 - val_loss: 0.1142 - val_accuracy: 0.9545
Epoch 116/1000
51/51 [==============================] - 0s 209us/step - loss: 0.0294 - accuracy: 1.0000 - val_loss: 0.1272 - val_accuracy: 0.9091
Epoch 117/1000
51/51 [==============================] - 0s 208us/step - loss: 0.0294 - accuracy: 1.0000 - val_loss: 0.1390 - val_accuracy: 0.8636
Epoch 118/1000
51/51 [==============================] - 0s 185us/step - loss: 0.0298 - accuracy: 1.0000 - val_loss: 0.1512 - val_accuracy: 0.8636
Epoch 119/1000
51/51 [==============================] - 0s 134us/step - loss: 0.0311 - accuracy: 1.0000 - val_loss: 0.1557 - val_accuracy: 

51/51 [==============================] - 0s 249us/step - loss: 0.0184 - accuracy: 1.0000 - val_loss: 0.1721 - val_accuracy: 0.8636
Epoch 170/1000
51/51 [==============================] - 0s 184us/step - loss: 0.0176 - accuracy: 1.0000 - val_loss: 0.1741 - val_accuracy: 0.8636
Epoch 171/1000
51/51 [==============================] - 0s 186us/step - loss: 0.0173 - accuracy: 1.0000 - val_loss: 0.1698 - val_accuracy: 0.8636
Epoch 172/1000
51/51 [==============================] - 0s 217us/step - loss: 0.0170 - accuracy: 1.0000 - val_loss: 0.1622 - val_accuracy: 0.8636
Epoch 173/1000
51/51 [==============================] - 0s 217us/step - loss: 0.0174 - accuracy: 1.0000 - val_loss: 0.1503 - val_accuracy: 0.8636
Epoch 174/1000
51/51 [==============================] - 0s 242us/step - loss: 0.0173 - accuracy: 1.0000 - val_loss: 0.1502 - val_accuracy: 0.8636
Epoch 175/1000
51/51 [==============================] - 0s 247us/step - loss: 0.0170 - accuracy: 1.0000 - val_loss: 0.1610 - val_accuracy: 

51/51 [==============================] - 0s 149us/step - loss: 0.0093 - accuracy: 1.0000 - val_loss: 0.2068 - val_accuracy: 0.8636
Epoch 225/1000
51/51 [==============================] - 0s 179us/step - loss: 0.0095 - accuracy: 1.0000 - val_loss: 0.2034 - val_accuracy: 0.8636
Epoch 226/1000
51/51 [==============================] - 0s 137us/step - loss: 0.0092 - accuracy: 1.0000 - val_loss: 0.2117 - val_accuracy: 0.8636
Epoch 227/1000
51/51 [==============================] - 0s 180us/step - loss: 0.0089 - accuracy: 1.0000 - val_loss: 0.2274 - val_accuracy: 0.8636
Epoch 228/1000
51/51 [==============================] - 0s 185us/step - loss: 0.0088 - accuracy: 1.0000 - val_loss: 0.2465 - val_accuracy: 0.8636
Epoch 229/1000
51/51 [==============================] - 0s 213us/step - loss: 0.0088 - accuracy: 1.0000 - val_loss: 0.2590 - val_accuracy: 0.8636
Epoch 230/1000
51/51 [==============================] - 0s 286us/step - loss: 0.0091 - accuracy: 1.0000 - val_loss: 0.2638 - val_accuracy: 

...

Epoch 775/1000
51/51 [==============================] - 0s 208us/step - loss: 2.7850e-04 - accuracy: 1.0000 - val_loss: 0.4903 - val_accuracy: 0.8636
Epoch 776/1000
51/51 [==============================] - 0s 185us/step - loss: 2.7842e-04 - accuracy: 1.0000 - val_loss: 0.4907 - val_accuracy: 0.8636
Epoch 777/1000
51/51 [==============================] - 0s 188us/step - loss: 2.7617e-04 - accuracy: 1.0000 - val_loss: 0.4938 - val_accuracy: 0.8636
Epoch 778/1000
51/51 [==============================] - 0s 216us/step - loss: 2.7522e-04 - accuracy: 1.0000 - val_loss: 0.4963 - val_accuracy: 0.8636
Epoch 779/1000
51/51 [==============================] - 0s 211us/step - loss: 2.7671e-04 - accuracy: 1.0000 - val_loss: 0.4984 - val_accuracy: 0.8636
Epoch 780/1000
51/51 [==============================] - 0s 198us/step - loss: 2.7357e-04 - accuracy: 1.0000 - val_loss: 0.4974 - val_accuracy: 0.8636
Epoch 781/1000
51/51 [==============================] - 0s 223us/step - loss: 2.7294e-04 - accuracy:

51/51 [==============================] - 0s 290us/step - loss: 2.3339e-04 - accuracy: 1.0000 - val_loss: 0.5095 - val_accuracy: 0.8636
Epoch 830/1000
51/51 [==============================] - 0s 284us/step - loss: 2.3246e-04 - accuracy: 1.0000 - val_loss: 0.5101 - val_accuracy: 0.8636
Epoch 831/1000
51/51 [==============================] - 0s 279us/step - loss: 2.3176e-04 - accuracy: 1.0000 - val_loss: 0.5116 - val_accuracy: 0.8636
Epoch 832/1000
51/51 [==============================] - 0s 285us/step - loss: 2.3150e-04 - accuracy: 1.0000 - val_loss: 0.5128 - val_accuracy: 0.8636
Epoch 833/1000
51/51 [==============================] - 0s 368us/step - loss: 2.3049e-04 - accuracy: 1.0000 - val_loss: 0.5124 - val_accuracy: 0.8636
Epoch 834/1000
51/51 [==============================] - 0s 262us/step - loss: 2.2940e-04 - accuracy: 1.0000 - val_loss: 0.5108 - val_accuracy: 0.8636
Epoch 835/1000
51/51 [==============================] - 0s 567us/step - loss: 2.2941e-04 - accuracy: 1.0000 - val_l

51/51 [==============================] - 0s 257us/step - loss: 1.9931e-04 - accuracy: 1.0000 - val_loss: 0.5427 - val_accuracy: 0.8636
Epoch 884/1000
51/51 [==============================] - 0s 240us/step - loss: 1.9946e-04 - accuracy: 1.0000 - val_loss: 0.5438 - val_accuracy: 0.8636
Epoch 885/1000
51/51 [==============================] - 0s 269us/step - loss: 1.9903e-04 - accuracy: 1.0000 - val_loss: 0.5435 - val_accuracy: 0.8636
Epoch 886/1000
51/51 [==============================] - 0s 467us/step - loss: 1.9822e-04 - accuracy: 1.0000 - val_loss: 0.5424 - val_accuracy: 0.8636
Epoch 887/1000
51/51 [==============================] - 0s 491us/step - loss: 1.9681e-04 - accuracy: 1.0000 - val_loss: 0.5391 - val_accuracy: 0.8636
Epoch 888/1000
51/51 [==============================] - 0s 405us/step - loss: 1.9587e-04 - accuracy: 1.0000 - val_loss: 0.5343 - val_accuracy: 0.8636
Epoch 889/1000
51/51 [==============================] - 0s 312us/step - loss: 1.9495e-04 - accuracy: 1.0000 - val_l

Epoch 938/1000
51/51 [==============================] - 0s 202us/step - loss: 1.6937e-04 - accuracy: 1.0000 - val_loss: 0.5367 - val_accuracy: 0.8636
Epoch 939/1000
51/51 [==============================] - 0s 184us/step - loss: 1.6923e-04 - accuracy: 1.0000 - val_loss: 0.5348 - val_accuracy: 0.8636
Epoch 940/1000
51/51 [==============================] - 0s 199us/step - loss: 1.6768e-04 - accuracy: 1.0000 - val_loss: 0.5356 - val_accuracy: 0.8636
Epoch 941/1000
51/51 [==============================] - 0s 239us/step - loss: 1.6786e-04 - accuracy: 1.0000 - val_loss: 0.5350 - val_accuracy: 0.8636
Epoch 942/1000
51/51 [==============================] - 0s 176us/step - loss: 1.6633e-04 - accuracy: 1.0000 - val_loss: 0.5365 - val_accuracy: 0.8636
Epoch 943/1000
51/51 [==============================] - 0s 257us/step - loss: 1.6603e-04 - accuracy: 1.0000 - val_loss: 0.5386 - val_accuracy: 0.8636
Epoch 944/1000
51/51 [==============================] - 0s 282us/step - loss: 1.6597e-04 - accuracy:

51/51 [==============================] - 0s 235us/step - loss: 1.4551e-04 - accuracy: 1.0000 - val_loss: 0.5471 - val_accuracy: 0.8636
Epoch 993/1000
51/51 [==============================] - 0s 234us/step - loss: 1.4526e-04 - accuracy: 1.0000 - val_loss: 0.5471 - val_accuracy: 0.8636
Epoch 994/1000
51/51 [==============================] - 0s 622us/step - loss: 1.4503e-04 - accuracy: 1.0000 - val_loss: 0.5479 - val_accuracy: 0.8636
Epoch 995/1000
51/51 [==============================] - 0s 352us/step - loss: 1.4422e-04 - accuracy: 1.0000 - val_loss: 0.5501 - val_accuracy: 0.8636
Epoch 996/1000
51/51 [==============================] - 0s 391us/step - loss: 1.4409e-04 - accuracy: 1.0000 - val_loss: 0.5524 - val_accuracy: 0.8636
Epoch 997/1000
51/51 [==============================] - 0s 375us/step - loss: 1.4383e-04 - accuracy: 1.0000 - val_loss: 0.5539 - val_accuracy: 0.8636
Epoch 998/1000
51/51 [==============================] - 0s 415us/step - loss: 1.4379e-04 - accuracy: 1.0000 - val_l

In [126]:
acc_test_comb3 = model2_comb2.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

22/22 [==============================] - 0s 185us/step
combination test accuracy: 86.36%


In [63]:
############## Random Forest ##############

In [64]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [65]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 60.24%


In [66]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [67]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 85.71%


In [68]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.48717949 0.46153846 0.66666667 0.64102564 0.43243243]
0.5377685377685377


In [69]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.90909091 1.         1.         0.88888889 0.875     ]
0.9345959595959595
